<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2021/examples/blob/main/example-map-reduce/colab/hadoop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install java first
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
#install hadoop. But we clean up first 
!rm -rf hadoop-3.3.0.tar.gz hadoop-3.3.0 /usr/local/hadoop
#download
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz
#untar
!tar -xzvf hadoop-3.3.0.tar.gz >/dev/null
!mv hadoop-3.3.0 /usr/local/hadoop


--2021-03-06 18:43:40--  https://archive.apache.org/dist/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 500749234 (478M) [application/x-gzip]
Saving to: ‘hadoop-3.3.0.tar.gz’

hadoop-3.3.0.tar.gz 100%[===================>] 477.55M  26.4MB/s    in 19s     

2021-03-06 18:44:00 (25.0 MB/s) - ‘hadoop-3.3.0.tar.gz’ saved [500749234/500749234]



In [3]:
#get the data file
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!gunzip title.basics.tsv.gz

--2021-03-06 18:44:10--  https://datasets.imdbws.com/title.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 65.8.246.41, 65.8.246.97, 65.8.246.4, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|65.8.246.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 136706444 (130M) [binary/octet-stream]
Saving to: ‘title.basics.tsv.gz’

title.basics.tsv.gz 100%[===================>] 130.37M  51.7MB/s    in 2.5s    

2021-03-06 18:44:13 (51.7 MB/s) - ‘title.basics.tsv.gz’ saved [136706444/136706444]



In [4]:
#check path and java home
!echo $JAVA_HOME
os.environ["PATH"]+= os.pathsep + "/usr/local/hadoop/bin"
!echo $PATH

/usr/lib/jvm/java-8-openjdk-amd64
/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin:/usr/local/hadoop/bin


In [ ]:
#check hadoop execution and ensure its in path
!/usr/local/hadoop/bin/hadoop

The hadoop process requires a mapper file and reducer file.

In [6]:
#the magic command with two %% will take the code in cell and copy to a file.
%%file mapper.py
#!/usr/bin/python2.7
import sys

def map_function(title):
    fields = title.strip().split('\t')         # Split title to fields using the data delimeter
    primaryTitle = fields[2]                   # Select the required field
    for word in primaryTitle.strip().split():  # Split primary title by words
        yield word.strip('!"*#$').lower(), 1                          # Use a word as a key

if __name__ == "__main__": 
  for line in sys.stdin:
      # Call map_function for each line in the input
      for key, value in map_function(line):
          # Emit key-value pairs using '|' as a delimeter  
          print(key + "|" + str(value))

Writing mapper.py


In [7]:
%%file reducer.py
#!/usr/bin/python2.7
import sys

prev_word = None
value_total = 0

if __name__ == "__main__": 
  for line in sys.stdin:          # For ever line in the input from stdin
      line = line.strip()         # Remove trailing characters
      word, value = line.split("|", 1)
      value = int(value)

      # Remember that Hadoop sorts mapper output by key, and the reducer takes these keys sorted
      if prev_word == word:
          value_total += value
      else:
          if prev_word != None:  # Write result to stdout
              print(prev_word + "|" + str(value_total))


          value_total = value
          prev_word = word

  if prev_word == word:  # Don't forget the last key/value pair
      print(prev_word + "|" + str(value_total))

Writing reducer.py


In [8]:
#give execute permission and clean output dir
!chmod +x mapper.py reducer.py
!rm -rf outputdir/

In [ ]:
#run hadoop
!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input title.basics.tsv -output outputdir -mapper mapper.py -reducer reducer.py -file mapper.py -file reducer.py 

In [13]:
# The output will be created in the outputdir - see the -output flag above
!ls -l outputdir/
!mv outputdir/part-00000 outputdir/hadoop-out.txt

total 16368
-rw-r--r-- 1 root root 16760258 Mar  6 18:46 part-00000
-rw-r--r-- 1 root root        0 Mar  6 18:46 _SUCCESS


In [14]:
#download file
from google.colab import files
files.download('outputdir/hadoop-out.txt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>